In [23]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [24]:
#df_All = pd.read_csv(r"C:\Users\saust\OneDrive - Sasol\1 Project rC4\Jupyter Notebooks\Report 10-20-23 No Fluff\df_All_Avg.csv")

df_All = pd.read_csv('df_Join_all.csv')


#df_All = pd.read_csv(r"C:\Users\saust\OneDrive\Desktop\CodeSpace DLs\102423\Project rC4\3 Final Machine Butanol\df_CDCA6.csv")


In [25]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [26]:
# # List of columns to exclude to run XGboost feature selection
# exclude_columns = ['Unnamed: 0', '%Al2O3_bM', 'Octanol PPM ', 'Hexanol PPM ',
#        'Ethanol PPM ', 'Dodecanol PPM ', 'Decanol PPM ', '%nC8OH', '%nC6OH', '%nC12OH', '%nC10OH ',
       
#        'TI52014', 'TI55013', 'TI55014', 'TI55015', 'TI55016', 'TI55017', 'TI55021', 'TI55023',
#        'TC52015', 'FC52018', 'II52554', 'TI40050', 'VI52558B'
#                    ]

# # Create a new DataFrame without the excluded columnsd
# df_All = df_All.drop(columns=exclude_columns)

In [27]:
# df_All = df_All[df_All['Date'] > '2022-06-15 00:00:00']

In [28]:

selected_columns = ['Butanol PPM', '425 %Al', 'TC55552', 'DI55102', 'FC55569', 'DI55152',
       'FC55576', 'TI55017', 'FC42428', 'AYC55580', 'LC55555', 'TI55023',
       'FFC55553', 'FC55555', 'FC55009', 'LC55557', 'LC55553', 'FC55003',
       'FFC55555'
#                      ^^^^ From XGB with all variables as continous ^^^^

#'Butanol PPM', <- Actuals ->
#                     '425 SAO Al', 'FFC55555', 'FC55003', 'FC55569',	
#                     'TC55555', 'LC55568', 'DI55152',	
#                     'FC55009', 'FC55102',	
#                     'LC55553', 'LC55557', 'LC90366'
                    ]

existing_columns = [col for col in selected_columns if col in df_All.columns]
df_All = df_All[existing_columns]

In [29]:
# First Iteration
# selected_columns = ['Butanol PPM', '425 SAO Al', 'FC42428', 'FC55003', 'FC55102', 'FC55152', 'FFC55555',
#        'LC55553', 'LC55557', 'LC55568', 'LC90366', 'TC55555']




In [30]:
df_All.columns

Index(['Butanol PPM', 'TC55552', 'DI55102', 'FC55569', 'DI55152', 'FC55576',
       'TI55017', 'FC42428', 'AYC55580', 'LC55555', 'TI55023', 'FFC55553',
       'FC55555', 'FC55009', 'LC55557', 'LC55553', 'FC55003', 'FFC55555'],
      dtype='object')

In [31]:
# Splitting into train and test
X = df_All.drop('Butanol PPM', axis=1)  # Assuming 'target' is your target column
y = df_All['Butanol PPM']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
#=============================================
# TRAIN & EVALUATE REGRESSION MODEL
#=============================================

#fit simple linear regression model
model = sm.OLS(y, X).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:            Butanol PPM   R-squared (uncentered):                   0.047
Model:                            OLS   Adj. R-squared (uncentered):              0.041
Method:                 Least Squares   F-statistic:                              8.009
Date:                Thu, 30 Nov 2023   Prob (F-statistic):                    4.43e-20
Time:                        09:19:05   Log-Likelihood:                         -21281.
No. Observations:                2770   AIC:                                  4.260e+04
Df Residuals:                    2753   BIC:                                  4.270e+04
Df Model:                          17                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [33]:
# Convert summary to DataFrame
summary_table = model.summary().tables[1]
summary_df = pd.DataFrame(summary_table.data[1:], columns=summary_table.data[0])

# Convert necessary columns to numeric
numeric_cols = ['coef', 'std err', 't', 'P>|t|', '[0.025', '0.975]']
summary_df[numeric_cols] = summary_df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Apply the filter for insignificance and CI crossing zero
threshold = 0.05
filtered_df = summary_df[(summary_df['P>|t|'] > threshold) & ((summary_df['[0.025'] < 0) & (summary_df['0.975]'] > 0))]

# Extracting the feature names
insignificant_features = filtered_df.iloc[:, 0].tolist()  # First column contains feature names
print(insignificant_features)


['TC55552', 'DI55102', 'FC55569', 'DI55152', 'FC55576', 'TI55017', 'FC42428', 'AYC55580', 'TI55023', 'FFC55553', 'FC55555', 'FC55009', 'LC55557', 'FFC55555']


In [34]:
print(filtered_df)

                   coef  std err      t  P>|t|    [0.025    0.975]
0    TC55552    -1.2761    1.008 -1.265  0.206    -3.253     0.701
1    DI55102   165.5401  203.581  0.813  0.416  -233.647   564.728
2    FC55569     0.0483    0.034  1.432  0.152    -0.018     0.115
3    DI55152    56.5175  360.657  0.157  0.875  -650.668   763.703
4    FC55576     0.0518    0.048  1.082  0.279    -0.042     0.146
5    TI55017    -0.3527    1.730 -0.204  0.838    -3.745     3.039
6    FC42428     0.0004    0.004  0.111  0.912    -0.007     0.008
7   AYC55580     1.5350    2.400  0.640  0.522    -3.170     6.240
9    TI55023     0.4212    2.273  0.185  0.853    -4.035     4.878
10  FFC55553   424.3005  322.428  1.316  0.188  -207.924  1056.525
11   FC55555     0.0005    0.006  0.080  0.937    -0.011     0.012
12   FC55009     0.0122    0.024  0.499  0.618    -0.036     0.060
13   LC55557     7.9824    4.542  1.758  0.079    -0.923    16.888
16  FFC55555 -1128.0375  603.887 -1.868  0.062 -2312.154    56

In [35]:
# Filter for significant features
significant_df = summary_df[(summary_df['P>|t|'] <= threshold) & ((summary_df['[0.025'] > 0) | (summary_df['0.975]'] < 0))]
significant_features = significant_df.iloc[:, 0].tolist()
print("\nSignificant Features:")
print(significant_features)


Significant Features:
['LC55555', 'LC55553', 'FC55003']
